<h2>MnMCount escrito en scala. Jupyter</h2>

_Comenzamos por iniciar una SparkSession que es el objeto principal o la base a partir de la cual cuelga toda la funcionalidad de Apache Spark. Es similar al SparkContext de los RDD, pero en este caso, para trabajar con SparkSQL, los DataFrame y DataSet_

In [39]:
val spark = SparkSession
.builder
.appName("MnMCountScala")
.getOrCreate()

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@2697e5d1


In [40]:
val mnmfile = "mnm_dataset.csv"

mnmfile: String = mnm_dataset.csv


_Leemos el csv indicando (_ `.option("header","true")` _) header=true para que la primera entrada de cada columna sea tomada como cabecera **ya que pordefecto este valor es header=false**._

_Indicamos también que inferSchema=true (_ `.option("inferSchema","true")` _), es decir que se infiera del esquema (equiv. tabla) el tipo de dato que corresponde a cada columna, **ya que por defecto inferSchema=false**, en este caso lo que hace es tratar todos los datos como_ `StringType`

In [41]:
val mnmDF=spark.read.format("csv")
.option("header","true")
.option("inferSchema","true")
.load(mnmfile)

mnmDF: org.apache.spark.sql.DataFrame = [State: string, Color: string ... 1 more field]


_Esto ya no es más que una consulta típica de SQL_

In [42]:
val countMnMDF=mnmDF
.select("State","Color","Count")
.groupBy("State","Color")
.agg(count("Count")
     .alias("Total"))
.orderBy(desc("Total"))

countMnMDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [State: string, Color: string ... 1 more field]


_Recordemos que en virtud del **lazy evaluation** todavía no se ha ejecutado nada, sino que se han recogido las **transformations** en un linaje para que al meter en juego una **action** se ejecuten las directrices de dicho grafo. Recordemos que cada **transformation** transforma un dataframe en otro dataframe. Acordarse tambien que hay:_
    <ul>
    <li> **narrow transformations**--> Se ejecutan en cada nodo y dan por si mismas el resultado </li>
    <li>**wide transformations**--> Se ejecutan en cada nodo y luego han de convinarse para dar el resultado</li>
    </ul>

In [43]:
countMnMDF.show(60)
println(s"total Rows = ${countMnMDF.count()}")

+-----+------+-----+
|State| Color|Total|
+-----+------+-----+
|   CA|Yellow| 1807|
|   WA| Green| 1779|
|   OR|Orange| 1743|
|   TX| Green| 1737|
|   TX|   Red| 1725|
|   CA| Green| 1723|
|   CO|Yellow| 1721|
|   CA| Brown| 1718|
|   CO| Green| 1713|
|   NV|Orange| 1712|
|   TX|Yellow| 1703|
|   NV| Green| 1698|
|   AZ| Brown| 1698|
|   WY| Green| 1695|
|   CO|  Blue| 1695|
|   NM|   Red| 1690|
|   AZ|Orange| 1689|
|   NM|Yellow| 1688|
|   NM| Brown| 1687|
|   UT|Orange| 1684|
|   NM| Green| 1682|
|   UT|   Red| 1680|
|   AZ| Green| 1676|
|   NV|Yellow| 1675|
|   NV|  Blue| 1673|
|   WA|   Red| 1671|
|   WY|   Red| 1670|
|   WA| Brown| 1669|
|   NM|Orange| 1665|
|   WY|  Blue| 1664|
|   WA|Yellow| 1663|
|   WA|Orange| 1658|
|   CA|Orange| 1657|
|   NV| Brown| 1657|
|   CA|   Red| 1656|
|   CO| Brown| 1656|
|   UT|  Blue| 1655|
|   AZ|Yellow| 1654|
|   TX|Orange| 1652|
|   AZ|   Red| 1648|
|   OR|  Blue| 1646|
|   UT|Yellow| 1645|
|   OR|   Red| 1645|
|   CO|Orange| 1642|
|   TX| Brown

_Cerramos la SparkSession_

In [44]:
spark.stop()